# NBA Models

## Data Export
```
DB_FILE=${FANTASY_HOME}/nba_hist_20082009-20212022.scored.db
SEASONS="20152016 20162017 20172018 20182019 20192020 20202021 20212022"

# player
dumpdata.sc --seasons $SEASONS --progress $DB_FILE --no_teams \
    --stats "*" --target_calc_stats "*" \
    --player_team_stats "*" --opp_team_stats "*" \
    --hist_recent_games 5 --hist_recent_mode ma --current_extra is_home \
    --format parquet -f nba_player.pq

# team
dumpdata.sc --seasons $SEASONS --progress $DB_FILE --no_players \
    --stats "*" --target_stats pts win \
    --opp_team_stats "*" \
    --hist_recent_games 5 --hist_recent_mode ma --current_extra is_home \
    --format parquet -f nba_team.pq
```

In [ ]:
RANDOM_SEED = 1
TRAINING_TIME = 600
VALIDATION_SEASON = 20212022
RECENT_GAMES = 5
TRAINING_SEASONS = [20152016, 20162017, 20172018, 20182019, 20192020, 20202021]

In [ ]:
from datetime import datetime
import sys
sys.path.append("..")

from fantasy_py import PlayerOrTeam
from train_test import load_data, train_test, create_fantasy_model

TARGET = ("stat", "pts")
MODEL_NAME = "NBA-team-pts"
DATA_FILENAME = "nba_team.pq"
P_OR_T = PlayerOrTeam.TEAM
INCLUDE_POS = None

# TARGET = ("stat", "win")
# MODEL_NAME = "NBA-team-win"
# DATA_FILENAME = "nba_team.pq"
# P_OR_T = "T"
# INCLUDE_POS = None

# TARGET = ("calc", "dk_score")
# MODEL_NAME = "NBA-DK"
# DATA_FILENAME = "nba_player.pq"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = True

raw_df, tt_data, one_hot_stats = load_data(
    DATA_FILENAME,
    TARGET,
    VALIDATION_SEASON,
    seed=RANDOM_SEED,
    include_position=INCLUDE_POS,
)

assert one_hot_stats is None

for automl_type in ["tpot"]:  # , "autosk"]:
    dt_trained = datetime.now()

    model_filepath, performance = train_test(
        automl_type, MODEL_NAME, TARGET, tt_data, RANDOM_SEED, TRAINING_TIME, dt_trained
    )
    performance["season"] = VALIDATION_SEASON
    model = create_fantasy_model(
        MODEL_NAME,
        model_filepath,
        dt_trained,
        tt_data[0],
        TARGET,
        TRAINING_TIME,
        P_OR_T,
        RECENT_GAMES,
        automl_type,
        performance,
        TRAINING_SEASONS,
        seed=RANDOM_SEED,
        raw_df=raw_df,
    )
    model_filepath = model.dump(".".join([MODEL_NAME, TARGET[1], automl_type, "model"]))
    print(f"Model file saved to '{model_filepath}'")
